In [1]:
import os
import time
import io
import struct
import sys
import math

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# infile = 'giphy.gif'
infile = 'sample_1.gif'
os.path.exists(infile)

True

In [4]:
file_obj = open(infile, mode='rb')

In [5]:
# file_obj.close()

# Compare signature

In [6]:
sig = file_obj.read(6)
sig

b'GIF89a'

In [7]:
len(sig)

6

In [8]:
type(sig)

bytes

In [9]:
gif87a_sig = b'GIF87a'
gif89a_sig = b'GIF89a'

In [10]:
msg = f'Hi {str(gif89a_sig)}'
print(msg)

Hi b'GIF89a'


In [11]:
type(gif89a_sig)

bytes

In [12]:
gif89a_sig

b'GIF89a'

In [13]:
sig == gif89a_sig

True

In [14]:
sig == gif87a_sig

False

# Screen Descriptor

In [15]:
screen_desc = file_obj.read(7)

In [16]:
len(screen_desc)

7

In [17]:
type(screen_desc)

bytes

In [18]:
width = struct.unpack('<h', screen_desc[0:2])[0]
height = struct.unpack('<h', screen_desc[2:4])[0]
packed_field = screen_desc[4]
background_color_index = screen_desc[5]
# Pixel Aspect Ratio is normal not used because we've known the width and height.
# pixel_aspect_ratio = screen_desc[6]

## unpack Packed Fields

In [19]:
packed_field

145

In [20]:
type(packed_field)

int

In [21]:
bin(packed_field)

'0b10010001'

In [22]:
global_color_table_flag = (packed_field & 0b10000000) >> 7
global_color_table_flag, bin(global_color_table_flag)

(1, '0b1')

> Color Resolution - Number of bits per primary color available to the original image, minus 1.

I think that means the number of bits to store a color. E.g. true color (RGB) requires 24 bits (3 bytes). Yeah, it makes no sense to me right now because the maximum value for this field is 7 (3 bits all 1).

> This value represents the size of the entire palette from which the colors in the graphic were selected, not the number of colors actually used in the graphic.

I still have no sense what the specification is talking about. The maximum value for this field is 7 so the maximum size of the palette is 7 something?

> For example, if the value in this field is 3, then the palette of the original image had 4 bits per primary color available to create the image.

Now I am noticing the specification is meaning about `original image`. Are there multiple images out there we need to care about outside the image this GIF file is storing? Even you raise the maximum value to 8, we can't still store a RGB color format in 8 bits.

> This value should be set to indicate the richness of the original palette, even if not every color from the whole palette is available on the source machine.

No, that does help at all. Specification!!!

In [23]:
color_resolution = (packed_field & 0b01110000) >> 4
color_resolution, bin(color_resolution)

(1, '0b1')

> Sort Flag - Indicates whether the Global Color Table is sorted.

Simple enough.

> If the flag is set, the Global Color Table is sorted, in order of decreasing importance. Typically, the order would be descreasing frequency, with most frequent color first. This assists a decoder, with fewer available colors, in choosing the best subset of colors; the decoder may use an initial segment of the table to render the graphic.

Yeah, I think I got that.

In [24]:
sort_flag = (packed_field & 0b00001000) >> 3
sort_flag, bin(sort_flag)

(0, '0b0')

> Size of Global Color Table - If the Global Color Table Flag is set to 1, the value in this field is used to calculate the number of bytes contained in the Global Color Table. To determine that actual size of the color table, raise 2 to [the value of the field + 1].

The size of global table is `(field_value + 1) * 2)` bytes, isn't it? So the maximum size for global palette is 16 bytes. Huh, 16 colors or 16 bytes to store color values?

> Even if there is not Global Color Table specified, set this field according to the above formula so that decoders can choose the best graphics mode to display the stream in.

How the size of the palette related to the graphics mode?

In [25]:
global_color_table_size = (packed_field & 0b00000111) >> 0
global_color_table_size, bin(global_color_table_size)

(1, '0b1')

In [26]:
global_palette_num_bytes = 3 * (2**(global_color_table_size+1))
global_palette_num_bytes

12

# Global palette (if the flag is set)

In [27]:
global_palette_data = file_obj.read(global_palette_num_bytes)
len(global_palette_data)

12

In [28]:
global_palette_data[0]

255

In [29]:
global_palette = [[*global_palette_data[i:i+3]] for i in range(0, len(global_palette_data), 3)]
len(global_palette)

4

In [30]:
global_palette

[[255, 255, 255], [255, 0, 0], [0, 0, 255], [0, 0, 0]]

In [31]:
def show_palette(palette):
    palette_width = 16
    
    image = []
    row = []
    for i, rgb in enumerate(palette):
        row.append(rgb)
        
        if (i > 0) and ((i+1) % palette_width == 0):
            image.append(row)
            row = []
#     print(image)
#     print(len(image))
    plt.imshow(image)
    plt.show()

In [32]:
# show_palette(global_palette)

# Next is unknown block type

Read the first byte to identify which type of block

In [33]:
block_identifier = file_obj.read(1)[0]
block_identifier, hex(block_identifier), bin(block_identifier)

(33, '0x21', '0b100001')

In [2]:
from decoder import GIF

In [3]:
infile = 'sample_1.gif'
file_obj = open(infile, mode='rb')
gif = GIF(file_obj)

Starting to parse GraphicControlExtension at 25.
Starting to parse ImageDescriptorBlock at 33.
data type: <class 'bytes'>


In [4]:
gif.blocks

In [5]:
img_block = gif.blocks[1]

In [18]:
code_stream = []

remain_bits_from_current_byte = 0
num_bits = img_block.lzw_min_code_size + 1
remain_bits_for_this_value = num_bits

clear_code = 2 ** img_block.lzw_min_code_size
eoi_code = clear_code + 1

# index for compressed stream
i = 0

In [36]:
value = 0
remain_bits_for_this_value = num_bits
print(value, remain_bits_for_this_value)

0 3


In [37]:
if remain_bits_from_current_byte == 0:
    b = img_block.compressed_data[i]
    remain_bits_from_current_byte = 8
    i += 1
print(i, b, bin(b), remain_bits_from_current_byte)

1 0 0b0 2


In [38]:
if remain_bits_for_this_value >= remain_bits_from_current_byte:
    print(value, remain_bits_for_this_value, remain_bits_from_current_byte)

    value = value << (num_bits - remain_bits_for_this_value)
    value += b
    remain_bits_for_this_value -= remain_bits_from_current_byte
    remain_bits_from_current_byte = 0
    
    print(value, remain_bits_for_this_value, remain_bits_from_current_byte)
else:
    print(value, remain_bits_for_this_value, remain_bits_from_current_byte)

    value = value << (num_bits - remain_bits_for_this_value)
    temp_value = b >> (remain_bits_from_current_byte - remain_bits_for_this_value)
    value += temp_value
    remain_bits_from_current_byte -= remain_bits_for_this_value
    remain_bits_for_this_value = 0
    
    print(value, remain_bits_for_this_value, remain_bits_from_current_byte)

0 3 2
0 1 0


In [34]:
# subtract value from b
print(b, bin(b))
b = b << (8 - remain_bits_from_current_byte)
print(b, bin(b))
b = b & 0xff
print(b, bin(b))
b = b >> (8 - remain_bits_from_current_byte)
print(b, bin(b))

12 0b1100
768 0b1100000000
0 0b0
0 0b0


In [35]:
print(value)
if value == (2**(num_bits) - 1):
    num_bits += 1
    print(num_bits)
code_stream.append(value)
print(code_stream)

3
[4, 3]


In [30]:
value == eoi_code

False

The compressed image data is stored in bits. The LZW minimum code size helps to indicate how many bits does the first value take (min code size + 1 bits).

How do we know when to increase the number of bits required for the next value?

We increase when the code value equals `2 ** num_bits - 1`.

After we reset our code table, where do we get the values for the code table? The first code table values are directly mapped from the color index (size of color table).

In [16]:
code_stream = []

num_bits = img_block.lzw_min_code_size + 1

clear_code = 2 ** img_block.lzw_min_code_size
eoi_code = clear_code + 1

# index for compressed stream
i = 0
remain_bits_from_current_byte = 0

# every loop will retrieve a code and append it to the code_stream
while True and (i < len(img_block.compressed_data)):
    value = 0
    remain_bits_for_this_value = num_bits
    print('value:', value, 'num_bits:', num_bits)

    while remain_bits_for_this_value != 0:
        if remain_bits_from_current_byte == 0:
            b = img_block.compressed_data[i]
            remain_bits_from_current_byte = 8
            i += 1
        print('i:', i, 'b:', b, 'bin(b):', bin(b), 'remain_bits_from_current_byte:', remain_bits_from_current_byte)

        if remain_bits_for_this_value >= remain_bits_from_current_byte:
            print('value:', value, 'remain_bits_for_this_value:', remain_bits_for_this_value, 'remain_bits_from_current_byte:', remain_bits_from_current_byte)

            value = value << (num_bits - remain_bits_for_this_value)
            value += b
            remain_bits_for_this_value -= remain_bits_from_current_byte
            remain_bits_from_current_byte = 0

            print('value:', value, 'remain_bits_for_this_value:', remain_bits_for_this_value, 'remain_bits_from_current_byte:', remain_bits_from_current_byte)
        else:
            print('value:', value, 'remain_bits_for_this_value:', remain_bits_for_this_value, 'remain_bits_from_current_byte:', remain_bits_from_current_byte)

            value = value << remain_bits_for_this_value
            temp_value = b >> (remain_bits_from_current_byte - remain_bits_for_this_value)
            value += temp_value
            remain_bits_from_current_byte -= remain_bits_for_this_value
            remain_bits_for_this_value = 0

            print('value:', value, 'remain_bits_for_this_value:', remain_bits_for_this_value, 'remain_bits_from_current_byte:', remain_bits_from_current_byte)

        # subtract value from b
        print(b, bin(b))
        b = b << (8 - remain_bits_from_current_byte)
        print(b, bin(b))
        b = b & 0xff
        print(b, bin(b))
        b = b >> (8 - remain_bits_from_current_byte)
        print(b, bin(b))

    print('value:', value)
    if value >= (2**(num_bits) - 1):
        num_bits += 1
        print('num_bits:', num_bits)
    code_stream.append(value)
    print('code_stream:', code_stream)

value: 0 num_bits: 3
i: 0 b: 1 bin(b): 0b1 remain_bits_from_current_byte: 1
value: 0 remain_bits_for_this_value: 3 remain_bits_from_current_byte: 1
value: 1 remain_bits_for_this_value: 2 remain_bits_from_current_byte: 0
1 0b1
256 0b100000000
0 0b0
0 0b0
i: 1 b: 140 bin(b): 0b10001100 remain_bits_from_current_byte: 8
value: 1 remain_bits_for_this_value: 2 remain_bits_from_current_byte: 8
value: 6 remain_bits_for_this_value: 0 remain_bits_from_current_byte: 6
140 0b10001100
560 0b1000110000
48 0b110000
12 0b1100
value: 6
code_stream: [6]
value: 0 num_bits: 3
i: 1 b: 12 bin(b): 0b1100 remain_bits_from_current_byte: 6
value: 0 remain_bits_for_this_value: 3 remain_bits_from_current_byte: 6
value: 1 remain_bits_for_this_value: 0 remain_bits_from_current_byte: 3
12 0b1100
384 0b110000000
128 0b10000000
4 0b100
value: 1
code_stream: [6, 1]
value: 0 num_bits: 3
i: 1 b: 4 bin(b): 0b100 remain_bits_from_current_byte: 3
value: 0 remain_bits_for_this_value: 3 remain_bits_from_current_byte: 3
value:

In [17]:
i

22

In [18]:
eoi_code

5